### Entraînement d'un Small BERT

* Lien Tf Hub : https://tfhub.dev/google/collections/bert/1

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text # Utilisé en sous-jacent lors du téléchargement du modèle 

from official.nlp import optimization 

tf.get_logger().setLevel('ERROR') # Pour éviter que les messages de warning apparaîssent 

### Construction du modèle 

In [2]:
### Liste de tous les BERT et ses variants (issus de Tf Hub)

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}


def get_model(bert_model_name):
    """ 
    Cette fonction prend un nom parmis la liste donnée de Tensor Hub et retourne le
    modèle BERT et son tokenizer associé.
    """

    tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]
    tfhub_encoder = map_name_to_handle[bert_model_name]

    return tfhub_handle_preprocess, tfhub_encoder

In [3]:
def build_model(preprocess_model_ref, encoder_ref):
    """
    Prend en paramètre un BERT_Tokenizer et un modèle BERT et ajoute deux couches Denses en 
    sortie (Dense(30) puis Dense(1)) pour l'adaptation à la classification de sentiment 
    positifs / négatifs
    """
    
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(preprocess_model_ref, name='preprocess')

    encoder_inputs = preprocessing_layer(text_input)
    print(f"Encoder input {encoder_ref}")
    encoder = hub.KerasLayer(encoder_ref, name='encoder')

    outputs = encoder(encoder_inputs)

    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(30, activation='relu', name='dense')(net)   # Ajout
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net) # Ajout 
    
    return tf.keras.Model(text_input, net)

### Téléchargement des ensembles de données 

In [5]:
# Attention au chemins d'accès 
seed = 1
batch_size = 32 

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    '/Users/erwan/Programmes/Stage/Text_Classification/aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    '/Users/erwan/Programmes/Stage/Text_Classification/aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    '/Users/erwan/Programmes/Stage/Text_Classification/aclImdb/test',
    batch_size=batch_size,
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2022-06-30 16:06:58.341552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Récupération du modèle 

In [9]:
config = {
    'init_learning_rate': 1e-4,
    'epochs': 5,
    'batch_size': batch_size,
}

steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()

num_train_steps = steps_per_epoch * config['epochs']
num_warmup_steps = int(0.1 * num_train_steps)

config['num_train_steps'] = num_train_steps
config['num_warmup_steps'] =  num_warmup_steps

In [10]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.metrics.BinaryAccuracy()

# Optimisateur utilsé pour BERT, j'ai choisi de le reprendre 
optimizer = optimization.create_optimizer(
    init_lr=config['init_learning_rate'],
    num_train_steps=config['num_train_steps'],
    num_warmup_steps=config['num_warmup_steps'],
    optimizer_type='adamw'
)

In [7]:
bert_model_name = 'small_bert/bert_en_uncased_L-2_H-128_A-2' 
prepro_model, encoder_model = get_model(bert_model_name=bert_model_name)

Encoder input https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1


In [ ]:
small_bert_model = build_model(
    preprocess_model_ref=prepro_model,
    encoder_ref=encoder_model,
)

small_bert_model.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=metric
)

# small_bert_model.summary()

__Remarquer que seuls les poids de la couche Dense sont entrainables.__ : ok !

In [11]:
# Pour TensorBoard
num_train = '001'
logdir = "logs/scalars/" + num_train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [12]:
print(f'BERT modèle : {encoder_model}')

history = small_bert_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config['epochs'],
    callbacks=[tensorboard_callback],
)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1
Epoch 1/5
625/625 [==============================] - 94s 147ms/step - loss: 0.6909 - binary_accuracy: 0.5271 - val_loss: 0.6637 - val_binary_accuracy: 0.5658
Epoch 2/5
625/625 [==============================] - 97s 155ms/step - loss: 0.6657 - binary_accuracy: 0.5631 - val_loss: 0.6513 - val_binary_accuracy: 0.5860
Epoch 3/5
625/625 [==============================] - 97s 156ms/step - loss: 0.6577 - binary_accuracy: 0.5687 - val_loss: 0.6465 - val_binary_accuracy: 0.5978
Epoch 4/5
625/625 [==============================] - 92s 147ms/step - loss: 0.6540 - binary_accuracy: 0.5792 - val_loss: 0.6443 - val_binary_accuracy: 0.6022
Epoch 5/5
625/625 [==============================] - 95s 153ms/step - loss: 0.6511 - binary_accuracy: 0.5806 - val_loss: 0.6431 - val_binary_accuracy: 0.5956


Résultats particulièrement mauvais..

In [13]:
# Pour afficher dans TensorBoard exécuter la commande suivante dans le terminal (ou avec %...)
# tensorboard --logir log/scalars

### Exportation du modèle 


In [ ]:
# Enregistrement 
save_model_path = './'

small_bert_model.save(save_model_path + num_train, include_optimizer=False)

In [55]:
# Récupération  
reloaded_model = tf.saved_model.load(save_model_path + num_train)

In [56]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'this is such an amazing movie!',  # this is the same sentence tried earlier
    'The movie was great!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was terrible...'
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(small_bert_model(tf.constant(examples)))

print('Results from the saved model:')
print_my_examples(examples, reloaded_results)
print('Results from the model in memory:')
print_my_examples(examples, original_results)

Results from the saved model:
input: this is such an amazing movie! : score: 0.821224
input: The movie was great!           : score: 0.616806
input: The movie was meh.             : score: 0.650367
input: The movie was okish.           : score: 0.507615
input: The movie was terrible...      : score: 0.262914

Results from the model in memory:
input: this is such an amazing movie! : score: 0.821224
input: The movie was great!           : score: 0.616806
input: The movie was meh.             : score: 0.650367
input: The movie was okish.           : score: 0.507615
input: The movie was terrible...      : score: 0.262914

